In [2]:
#%run -i '../../scripts/utility_readblt.py' 'emu'

In [1]:
from utility_common import *
from utility_plotter import *
import ROOT
from IPython.display import clear_output
## add plotting option and load plotting parameter table
%matplotlib inline
#%qtconsole --style monokai
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

Welcome to JupyROOT 6.13/02


# 1. Read data and MC as dataframe

In [3]:
selection  = "emu"
nbjetcut   =  ">=1"

pickledir  =  "../../data/pickle/{}/".format(selection)

cuts = GetSelectionCut(selection) + "& (nBJets{})".format(nbjetcut)

Data = LoadDataframe(pickledir + "data2016").query(cuts)
MCzz = LoadDataframe(pickledir + "mcdiboson").query(cuts)
MCdy = LoadDataframe(pickledir + "mcdy").query(cuts)
MCt  = LoadDataframe(pickledir + "mct").query(cuts)
MCtt = LoadDataframe(pickledir + "mctt").query(cuts)

In [4]:
#MCzz.eventWeight = 0.97*MCzz.eventWeight
#MCdy.eventWeight = 0.97*MCdy.eventWeight
#MCt .eventWeight = 0.97*MCt .eventWeight
#MCtt.eventWeight = 0.97*MCtt.eventWeight

In [5]:
Data = Data.drop_duplicates(subset=['runNumber', 'evtNumber'])

In [6]:
# signal mc is tt+tW
MCsg = pd.concat([MCt,MCtt],ignore_index=True)

MCsgf = MCsg.query('(genCategory >= 16) ') # semilep and had
MCsg0 = MCsg.query('genCategory in [1,2,4,5,6,7,8,9]') #other lep
MCsg1 = MCsg.query('genCategory in [3]') #ee
MCsg2 = MCsg.query('genCategory in [10,11,12]') #etau
MCsg3 = MCsg.query('genCategory in [13,14,15]') #mutau


df_list = [MCzz,MCdy,MCsgf,MCsg0,MCsg1,MCsg2,MCsg3,Data]

# 2. Plot kinematics

In [7]:
pp = pd.read_csv("plotparameters.csv")
label_list = ["Diboson","V+Jets",r'$tt/tW \rightarrow l + h$ (other)',
              r'$tt/tW \rightarrow l + l$ (other) ',
              r'$tt/tW \rightarrow e + \mu$', 
              r'$tt/tW \rightarrow e + \tau$',
              r'$tt/tW \rightarrow \mu + \tau$',
              'data']
color_list = ["#a32020","#e0301e","#eb8c00","gold","#49feec","deepskyblue","mediumpurple","k"]
plotoutdir = GetPlotDir(selection, nbjetcut)
print(plotoutdir)

/home/zchen/Documents/Analysis/workplace/plot/emu/combined/


In [9]:
for index, row in pp.iterrows():
    v,a,b,step,xl = row["var"],row["lower"],row["upper"],row["step"],row["xlabel"]
    plotter = SelectionPlotter(v,a,b,step,df_list)
    plotter.settingPlot(xl,label_list,color_list)
    plotter.makePlot(plotoutdir)
    # print out process
    print("making plots: {0}/{1}".format(index,len(pp)))
    clear_output(wait=True)
    plt.close()
print('finish all making plots')

finish all making plots


# 3. Counting

In [13]:
ctr = SelectionCounter()

In [14]:
print("(nBJets{})".format(nbjetcut))

print("MCtt: {}".format(ctr.countSlt_ByTauDecay(MCtt,withweights=False)) )
print("MCtt width: {}".format(ctr.countSlt_ByTauDecay(MCtt,withweights=True)) )

print("MCt : {}".format(ctr.countSlt_ByTauDecay(MCt, withweights=False)) )
print("MCt width: {}".format(ctr.countSlt_ByTauDecay(MCt, withweights=True)) )

(nBJets>1)
MCtt: [1, 7, 21356, 0, 0, 297, 0, 4, 0, 0, 1278, 1, 4970, 1, 44, 8, 143, 2, 11, 0, 0]
MCtt width: [1.1140595364570618, 6.488890656828881, 19223.172941857065, 0.0, 0.0, 265.0999906659173, 0.0, 3.7237268424034116, 0.0, 0.0, 1136.2771999710694, 1.067586166858673, 4459.322526598808, 0.8062084996700286, 36.50298745946027, 6.876936626732349, 124.49208696966991, 1.7995477509498596, 9.758677277565003, 0.0, 0.0]
MCt : [0, 0, 71, 0, 0, 4, 0, 0, 0, 0, 4, 0, 22, 0, 0, 0, 1, 0, 0, 0, 0]
MCt width: [0.0, 0.0, 60.6827909053676, 0.0, 0.0, 3.45623785495758, 0.0, 0.0, 0.0, 0.0, 4.056594445109368, 0.0, 20.425029012709857, 0.0, 0.0, 0.0, 0.7272240996360778, 0.0, 0.0, 0.0, 0.0]


In [23]:
ctr.countSlt_Scaled(df_list)

data:24911.0
TotalMC:24836.378190521296
------ MC break down ------
[1.72660693e+01 1.75386282e+01 4.79352392e+01 1.16117096e+02
 2.15726687e+04 1.36827907e+03 1.69657334e+03]


In [24]:
for thisdf in [MCzz, MCdy, MCt, MCtt, Data]:
    n = np.sum(thisdf.eventWeight)
    nerr = np.sum(thisdf.eventWeight**2)**0.5
    print('{:.2f},{:.2f}'.format(n,nerr))
#mc,h = PlotHistogram("lepton2_pt",0,150,5,"lepton2_pt [GeV]", df_list,label_list,color_list)

17.27,4.03
17.54,3.55
666.58,29.10
24134.99,66.90
24911.00,157.83


In [14]:
# print Generator information
f = ROOT.TFile("../../data/root/2016MC.root")
print("MCtt : {}".format(ctr.countGen_ByTauDecay(f,'ttbar_inclusive')) )
temp = np.array(ctr.countGen_ByTauDecay(f,'t_tw'))
temp+= np.array(ctr.countGen_ByTauDecay(f,'tbar_tw'))
print("MCt  : {}".format(list(temp)) )

MCtt : [1811409.0, 1811532.0, 3620281.0, 57057.0, 54227.0, 111125.0, 418126.0, 418126.0, 763147.0, 642646.0, 626108.0, 2350887.0, 643368.0, 627277.0, 2353727.0, 22670017.0, 22653517.0, 4016590.0, 3923504.0, 14718933.0, 70930278.0]
MCt  : [23105.0, 23040.0, 46342.0, 694.0, 717.0, 1420.0, 5260.0, 5260.0, 9727.0, 8199.0, 8029.0, 30054.0, 8163.0, 8006.0, 29970.0, 290101.0, 289467.0, 50864.0, 50050.0, 187515.0, 904067.0]
